In [ ]:
import json
import tkinter as tk
from tkinter import messagebox


class Task:
    def __init__(self, title, description, category):
        self.title = title
        self.description = description
        self.category = category
        self.completed = False

    def mark_completed(self):
        self.completed = True

    def to_dict(self):
        return {
            "title": self.title,
            "description": self.description,
            "category": self.category,
            "completed": self.completed
        }

    @classmethod
    def from_dict(cls, data):
        title = data.get("title", "Untitled Task")
        description = data.get("description", "")
        category = data.get("category", "General")
        completed = data.get("completed", False)
        
        task = cls(title, description, category)
        task.completed = completed
        return task


def save_tasks(tasks):
    try:
        with open('tasks.json', 'w') as f:
            json.dump([task.to_dict() for task in tasks], f, indent=4)
    except Exception as e:
        messagebox.showerror("Error", f"Error saving tasks: {e}")


def load_tasks():
    try:
        with open('tasks.json', 'r') as f:
            return [Task.from_dict(data) for data in json.load(f)]
    except FileNotFoundError:
        return []
    except json.JSONDecodeError:
        messagebox.showerror("Error", "Error reading tasks. Starting with an empty task list.")
        return []


class ToDoApp:
    def __init__(self, root):
        self.root = root
        self.root.title("To-Do List Manager")
        self.tasks = load_tasks()
        
        # GUI Elements
        self.frame = tk.Frame(root)
        self.frame.pack(pady=20)

        self.title_label = tk.Label(self.frame, text="Task Title:")
        self.title_label.grid(row=0, column=0)

        self.title_entry = tk.Entry(self.frame)
        self.title_entry.grid(row=0, column=1)

        self.desc_label = tk.Label(self.frame, text="Task Description:")
        self.desc_label.grid(row=1, column=0)

        self.desc_entry = tk.Entry(self.frame)
        self.desc_entry.grid(row=1, column=1)

        self.cat_label = tk.Label(self.frame, text="Category:")
        self.cat_label.grid(row=2, column=0)

        self.cat_entry = tk.Entry(self.frame)
        self.cat_entry.grid(row=2, column=1)

        self.add_button = tk.Button(self.frame, text="Add Task", command=self.add_task)
        self.add_button.grid(row=3, column=0, columnspan=2, pady=10)

        self.task_listbox = tk.Listbox(root, height=10, width=50)
        self.task_listbox.pack(pady=10)

        self.complete_button = tk.Button(root, text="Mark as Completed", command=self.mark_task_completed)
        self.complete_button.pack(pady=5)

        self.delete_button = tk.Button(root, text="Delete Task", command=self.delete_task)
        self.delete_button.pack(pady=5)

        self.save_button = tk.Button(root, text="Save Tasks", command=self.save_tasks)
        self.save_button.pack(pady=10)

        self.update_task_listbox()

    def add_task(self):
        title = self.title_entry.get()
        description = self.desc_entry.get()
        category = self.cat_entry.get()

        if title:
            new_task = Task(title, description, category)
            self.tasks.append(new_task)
            self.update_task_listbox()
            self.title_entry.delete(0, tk.END)
            self.desc_entry.delete(0, tk.END)
            self.cat_entry.delete(0, tk.END)
            messagebox.showinfo("Success", "Task added successfully!")
        else:
            messagebox.showerror("Error", "Task title is required!")

    def update_task_listbox(self):
        self.task_listbox.delete(0, tk.END)
        for i, task in enumerate(self.tasks):
            status = "Completed" if task.completed else "Pending"
            self.task_listbox.insert(tk.END, f"{i+1}. {task.title} [{task.category}] - {status}")

    def mark_task_completed(self):
        try:
            task_index = self.task_listbox.curselection()[0]
            self.tasks[task_index].mark_completed()
            self.update_task_listbox()
            messagebox.showinfo("Success", "Task marked as completed!")
        except IndexError:
            messagebox.showerror("Error", "Please select a task to mark as completed.")

    def delete_task(self):
        try:
            task_index = self.task_listbox.curselection()[0]
            del self.tasks[task_index]
            self.update_task_listbox()
            messagebox.showinfo("Success", "Task deleted successfully!")
        except IndexError:
            messagebox.showerror("Error", "Please select a task to delete.")

    def save_tasks(self):
        save_tasks(self.tasks)
        messagebox.showinfo("Success", "Tasks saved successfully!")


if __name__ == "__main__":
    root = tk.Tk()
    app = ToDoApp(root)
    root.mainloop()
